In [1]:
#Dependencies
from splinter import Browser
import pandas as pd
from bs4 import BeautifulSoup
import time
import requests
import pymongo

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Choose the executable path to driver 
#def init_browser():
executable_path = {'executable_path': '/Users/danieltaylor/Desktop/dstaylor37/11-Mission-To-Mars/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
def scrape():
    Browser = init_browser()

In [5]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [6]:
#HTML object
html = browser.html

#Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve the parent divs for all articles
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text

#display scrapped data
print(news_title)
print(news_p)

For InSight, Dust Cleanings Will Yield New Science
Wind can be crucial to clearing dust from spacecraft solar panels on Mars. With InSight's meteorological sensors, scientists get their first measurements of wind and dust interacting "live" on the Martian surface.  


In [7]:
### JPL Mars Space Images - Featured Image
###Visit the url for JPL Featured Space Image 
###[here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

image_url_featured = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url_featured)

In [8]:
#html object
html_image = browser.html

#parse html with beautiful soup
soup = BeautifulSoup(html_image, 'html.parser')

#retrieve background-image url from style tag
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

#website url
main_url = 'https://www.jpl.nasa.gov'

#combine the image and web site url
featured_image_url = main_url + featured_image_url

#show the combined image link
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19113-1920x1200.jpg'

In [9]:
###Mars Weather
#Scrape the latest Mars weather tweet from the page
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [10]:
html_weather = browser.html

weather_soup = BeautifulSoup(html_weather, 'html.parser')

mars_weather_tweet = weather_soup.find_all('div', class_='js-tweet-text-container')

for tweet in mars_weather_tweet:
    mars_weather_tweet = tweet.find('p').text
    if tweet.text.strip().startswith('InSight sol'):
        mars_weather = tweet.text.strip()

In [11]:
print(mars_weather_tweet)

Mars, I hear you. I’ve detected some quiet but distinct shaking on #Mars. The faint rumbles appear to have come from the inside of the planet, and are still being studied by my team. Take a listen.http://go.nasa.gov/2GCEBtm pic.twitter.com/Z8Hn03jigO


In [12]:
##Mars Fact
#Visit the Mars Facts webpage [here](https://space-facts.com/mars/) 
#and use Pandas to scrape the table containing facts about the planet including 
#Diameter, Mass, etc.

#Use Pandas to convert the data to a HTML table string.

url = 'https://space-facts.com/mars/'
browser.visit(url)

mars_df = pd.read_html(url)
mars_df = (mars_df[0])


mars_df.columns=['Description', 'Value']

mars_df.set_index('Description', inplace=True)

mars_df.to_html()

data = mars_df.to_dict(orient='records')

mars_df

# mars_df = mars_df.to_html(classes='mars')
# table_data = mars_df.replace('\n',' ')
# table_data

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


mars_df = mars_df.to_html(classes='mars')
table_data = mars_df.replace('\n\, ' ')
table_data

In [13]:
##Mars Hemispheres

hemi_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)

html_hemi = browser.html

soup = BeautifulSoup(html_hemi, 'html.parser')

items = soup.find_all('div', class_='item')

hiu = []

hemi_images_urls = 'https://astrogeology.usgs.gov'

for i in items:
    title = i.find('h3').text
    thumb_url = i.find('a', class_='itemLink product-item')['href']
    browser.visit(hemi_images_urls + thumb_url)
    thumb_image_html=browser.html
    soup = BeautifulSoup(thumb_image_html, 'html.parser')
    full_image_url = hemi_images_urls + soup.find('img', class_='wide-image')['src']
    hiu.append({'title': title, 'img_url': full_image_url})
hiu


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [28]:
browser.quit()